In [6]:
import sqlite3
from sqlite3 import Error
import pandas as pd

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# Replace with the path to the northwind.db file on your computer (e.g. "C:/Users/...")
db_path = 'G:/LighthouseLabs/Data/SQL/northwind.db'

con = create_connection(db_path)
cur = con.cursor()

Connection to SQLite DB successful


In [7]:
# Convenience function for queries. Note that "return_pandas" can only be set to True for SQL statements
# that return data (e.g. it has to be False for statements like DROP TABLE and CREATE TABLE)
def execute_query(query_string, return_pandas=True):
    if return_pandas:
        response = pd.read_sql_query(query_string, con)
    else:
        cur.execute(query_string)
        response = cur.fetchall()
    if response is None:
        con.commit()
    else:
        return response

In [8]:
execute_query("""select count(*) from orders;""")


,count(*)
0,830


In [9]:
execute_query("""select min(orderdate) from orders;""")

,min(orderdate)
0,1996-07-04


In [10]:
execute_query("""select max(orderdate) from orders;""")

,max(orderdate)
0,1998-05-06


In [12]:
execute_query("""DROP TABLE if exists end_obs_dates;""", return_pandas=False)
execute_query("""CREATE TABLE end_obs_dates 
AS

-- Recursive query example (something like FOR loop for SQL)
WITH RECURSIVE
  cnt(x) AS (
     SELECT 0
     UNION ALL
     SELECT x+1 FROM cnt
      LIMIT (SELECT ROUND(((julianday('1998-06-01') - julianday('1996-08-01'))/30) + 1)) -- this counts number of months between these two days
  )
SELECT date('1996-08-01', '+' || x || ' month') as end_obs_date FROM cnt;""", return_pandas=False)

[]

In [13]:
execute_query("""select * from end_obs_dates""")

,end_obs_date
0,1996-08-01
1,1996-09-01
2,1996-10-01
3,1996-11-01
4,1996-12-01
5,1997-01-01
6,1997-02-01
7,1997-03-01
8,1997-04-01
9,1997-05-01


In [14]:
execute_query("""DROP TABLE if exists ads_population_hist;""", return_pandas=False)
execute_query("""CREATE TABLE ads_population_hist 
AS
SELECT A.*,
       B.*
FROM end_obs_dates AS A
CROSS JOIN (SELECT DISTINCT customerid FROM customers) AS B
;""", return_pandas=False)

[]

In [15]:
execute_query("""select * from ads_population_hist limit 10""")

,end_obs_date,customerid
0,1996-08-01,ALFKI
1,1996-08-01,ANATR
2,1996-08-01,ANTON
3,1996-08-01,AROUT
4,1996-08-01,BERGS
5,1996-08-01,BLAUS
6,1996-08-01,BLONP
7,1996-08-01,BOLID
8,1996-08-01,BONAP
9,1996-08-01,BOTTM


In [16]:
execute_query("""SELECT *,
       unitprice*quantity AS totalprice_for_product
FROM "Order Details" LIMIT 20""")

,OrderID,ProductID,UnitPrice,Quantity,Discount,totalprice_for_product
0,10248,11,14.0,12,0.00,168.0
1,10248,42,9.8,10,0.00,98.0
2,10248,72,34.8,5,0.00,174.0
3,10249,14,18.6,9,0.00,167.4
4,10249,51,42.4,40,0.00,1696.0
5,10250,41,7.7,10,0.00,77.0
6,10250,51,42.4,35,0.15,1484.0
7,10250,65,16.8,15,0.15,252.0
8,10251,22,16.8,6,0.05,100.8
9,10251,57,15.6,15,0.05,234.0


In [17]:
execute_query("""SELECT A.orderid,
     COUNT(DISTINCT A.productid) AS no_of_distinct_products,
     SUM(A.quantity) AS no_of_items,
     SUM(A.totalprice_for_product) AS total_price 
     FROM (SELECT *,
                  unitprice*quantity AS totalprice_for_product
           FROM "Order Details") AS A
GROUP BY 1;""")

,OrderID,no_of_distinct_products,no_of_items,total_price
0,10248,3,27,440.0
1,10249,2,49,1863.4
2,10250,3,60,1813.0
3,10251,3,41,670.8
4,10252,3,105,3730.0
...,...,...,...,...
825,11073,2,30,300.0
826,11074,1,14,244.3
827,11075,3,42,586.0
828,11076,3,50,1057.0


In [22]:
execute_query("""CREATE TABLE ads_orders_hist 
AS
SELECT A.orderid,
       A.customerid,
       A.end_obs_date,
       B.no_of_distinct_products,
       B.no_of_items,
       B.total_price
FROM (
    SELECT orderid,
             customerid,
             orderdate,
             date(orderdate,'start of month','+1 month') as end_obs_date
    FROM orders)
AS A
LEFT OUTER JOIN (
    SELECT A.orderid,
         COUNT(DISTINCT A.productid) AS no_of_distinct_products,
         SUM(A.quantity) AS no_of_items,
         SUM(A.totalprice_for_product) AS total_price
    FROM (
        SELECT *,
            unitprice*quantity AS totalprice_for_product
        FROM "Order Details") 
    AS A
    GROUP BY 1) 
AS B 
ON A.orderid = B.orderid;

""", return_pandas=False)

OperationalError: table ads_orders_hist already exists

In [24]:
execute_query("""select orderid
    ,count(*)
from ads_orders_hist
group by 1
order by 2 desc
limit 5
;""")

,orderid,count(*)
0,11077,1
1,11076,1
2,11075,1
3,11074,1
4,11073,1


In [27]:
execute_query("""
create table ads_observation_hist as
select 
    A.*
    -- we can replace missings with 0 because it means there were no orders for this client during specific month.
    ,coalesce(B.no_of_distinct_orders_1M, 0) as no_of_distinct_orders_1M
    ,coalesce(B.no_of_items_1M, 0) as no_of_items_1M
    ,coalesce(B.total_price_1M, 0) as total_price_1M
from ads_population_hist as A
left outer join (
    -- we need to group by our orders to customer level
    select customerid
        ,end_obs_date
        ,count(distinct orderid) as no_of_distinct_orders_1M
        ,sum(no_of_items) as no_of_items_1M
        ,sum(total_price) as total_price_1M
    from ads_orders_hist
    group by 1,2
) as B
on A.customerid = B.customerid
  and A.end_obs_date = B.end_obs_date
;""", return_pandas = False)

[]

In [28]:
execute_query("""select customerid
    ,end_obs_date
    ,count(*)
from ads_observation_hist
group by 1,2
order by 3 desc
limit 5
;""")

,customerid,end_obs_date,count(*)
0,ALFKI,1996-08-01,1
1,ALFKI,1996-09-01,1
2,ALFKI,1996-10-01,1
3,ALFKI,1996-11-01,1
4,ALFKI,1996-12-01,1


In [30]:
execute_query("""select * from ads_observation_hist limit 10""")

,end_obs_date,customerid,no_of_distinct_orders_1M,no_of_items_1M,total_price_1M
0,1996-08-01,ALFKI,0,0,0.0
1,1996-08-01,ANATR,0,0,0.0
2,1996-08-01,ANTON,0,0,0.0
3,1996-08-01,AROUT,0,0,0.0
4,1996-08-01,BERGS,0,0,0.0
5,1996-08-01,BLAUS,0,0,0.0
6,1996-08-01,BLONP,1,50,1176.0
7,1996-08-01,BOLID,0,0,0.0
8,1996-08-01,BONAP,0,0,0.0
9,1996-08-01,BOTTM,0,0,0.0
